# TSA Chapter 2: Python Lab: Model Selection with AIC/BIC

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_Ch2/TSA_ch2_python_selection/TSA_ch2_python_selection.ipynb)

This notebook demonstrates:
- ADF stationarity test, compare AIC/BIC across AR(1), MA(1), ARMA(1,1), ARMA(2,1), select best model, generate forecasts with confidence intervals

In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from matplotlib.patches import Polygon
# Style configuration
COLORS = {
    'blue': '#1A3A6E',
    'red': '#DC3545',
    'green': '#2E7D32',
    'orange': '#E67E22',
    'gray': '#666666',
    'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none',
    'figure.facecolor': 'none',
    'savefig.transparent': True,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': False,
    'font.size': 9,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'legend.fontsize': 8,
    'figure.dpi': 150,
    'lines.linewidth': 1.2,
    'axes.edgecolor': '#333333',
    'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    """Save chart as PDF and PNG."""
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}.pdf + .png')

In [ ]:
np.random.seed(42)

print("=" * 60)
print("PYTHON EXERCISE 2: MODEL SELECTION")
print("=" * 60)

# Generate an ARMA(1,1) series for demonstration
ar_params = np.array([1, -0.7])
ma_params = np.array([1, 0.4])
arma_process = ArmaProcess(ar_params, ma_params)
y = arma_process.generate_sample(nsample=300)

# Step 1: Stationarity test
adf_result = adfuller(y)
print(f"\n1. ADF Test for Stationarity:")
print(f"   ADF Statistic: {adf_result[0]:.4f}")
print(f"   p-value: {adf_result[1]:.6f}")
print(f"   → {'Stationary' if adf_result[1] < 0.05 else 'Non-stationary'}")

# Step 2: Compare models
models = {
    'AR(1)':     (1, 0, 0),
    'MA(1)':     (0, 0, 1),
    'ARMA(1,1)': (1, 0, 1),
    'ARMA(2,1)': (2, 0, 1),
}

print(f"\n2. Model Comparison:")
print(f"   {'Model':<12} {'AIC':>10} {'BIC':>10}")
print("   " + "-" * 34)

results = {}
for name, order in models.items():
    try:
        fit = ARIMA(y, order=order).fit()
        results[name] = fit
        print(f"   {name:<12} {fit.aic:>10.2f} {fit.bic:>10.2f}")
    except Exception as e:
        print(f"   {name:<12} {'FAILED':>10}")

# Step 3: Select best model
best_aic = min(results, key=lambda k: results[k].aic)
best_bic = min(results, key=lambda k: results[k].bic)
print(f"\n3. Best Model:")
print(f"   By AIC: {best_aic} (AIC = {results[best_aic].aic:.2f})")
print(f"   By BIC: {best_bic} (BIC = {results[best_bic].bic:.2f})")

# Step 4: Forecast with best model
best_model = results[best_aic]
forecast = best_model.get_forecast(steps=20)
fc_mean = forecast.predicted_mean
fc_ci = forecast.conf_int(alpha=0.05)

print(f"\n4. Forecast (next 5 periods):")
for i in range(5):
    print(f"   h={i+1}: {fc_mean.iloc[i]:.4f} [{fc_ci.iloc[i, 0]:.4f}, {fc_ci.iloc[i, 1]:.4f}]")